In [29]:
# AlexNet 8 (Train-Test)

#coding=utf-8
from __future__ import print_function
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

start = time.clock() #计算开始时间

mnist = input_data.read_data_sets("/MNIST_data", one_hot=True)

# 定义网络超参数
learning_rate = 0.001
training_iters = 2000
batch_size = 64
display_step = 20

# 定义网络参数
n_input = 784 # 输入的维度
n_classes = 10 # 标签的维度
dropout = 0.8 # Dropout 的概率

# 占位符输入
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# 卷积操作
def conv2d(name, l_input, w, b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(l_input, w, strides=[1, 1, 1, 1], padding='SAME'),b), name=name)

# 最大下采样操作
def max_pool(name, l_input, k):
    return tf.nn.max_pool(l_input, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME', name=name)

# 归一化操作
def norm(name, l_input, lsize=4):
    return tf.nn.lrn(l_input, lsize, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name=name)

# 定义整个网络 
def alex_net(_X, _weights, _biases, _dropout):
    # 向量转为矩阵
    _X = tf.reshape(_X, shape=[-1, 28, 28, 1])

    # 卷积层
    conv1 = conv2d('conv1', _X, _weights['wc1'], _biases['bc1'])
    # 下采样层
    pool1 = max_pool('pool1', conv1, k=2)
    # 归一化层
    norm1 = norm('norm1', pool1, lsize=4)
    # Dropout
    norm1 = tf.nn.dropout(norm1, _dropout)

    # 卷积
    conv2 = conv2d('conv2', norm1, _weights['wc2'], _biases['bc2'])
    # 下采样
    pool2 = max_pool('pool2', conv2, k=2)
    # 归一化
    norm2 = norm('norm2', pool2, lsize=4)
    # Dropout
    norm2 = tf.nn.dropout(norm2, _dropout)

    # 卷积
    conv3 = conv2d('conv3', norm2, _weights['wc3'], _biases['bc3'])
    # 下采样
    pool3 = max_pool('pool3', conv3, k=2)
    # 归一化
    norm3 = norm('norm3', pool3, lsize=4)
    # Dropout
    norm3 = tf.nn.dropout(norm3, _dropout)

    # 全连接层，先把特征图转为向量
    dense1 = tf.reshape(norm3, [-1, _weights['wd1'].get_shape().as_list()[0]]) 
    dense1 = tf.nn.relu(tf.matmul(dense1, _weights['wd1']) + _biases['bd1'], name='fc1') 
    # 全连接层
    dense2 = tf.nn.relu(tf.matmul(dense1, _weights['wd2']) + _biases['bd2'], name='fc2') # Relu activation

    # 网络输出层
    out = tf.matmul(dense2, _weights['out']) + _biases['out']
    return out

# 存储所有的网络参数
weights = {
    'wc1': tf.Variable(tf.random_normal([3, 3, 1, 64])),
    'wc2': tf.Variable(tf.random_normal([3, 3, 64, 128])),
    'wc3': tf.Variable(tf.random_normal([3, 3, 128, 256])),
    'wd1': tf.Variable(tf.random_normal([4*4*256, 1024])),
    'wd2': tf.Variable(tf.random_normal([1024, 1024])),
    'out': tf.Variable(tf.random_normal([1024, 10]))
}
biases = {
    'bc1': tf.Variable(tf.random_normal([64])),
    'bc2': tf.Variable(tf.random_normal([128])),
    'bc3': tf.Variable(tf.random_normal([256])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'bd2': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# 构建模型
pred = alex_net(x, weights, biases, keep_prob)

# 定义损失函数和学习步骤
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred, labels = y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# 测试网络
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

config = tf.ConfigProto(
        device_count = {'GPU': 0}
    )

saver = tf.train.Saver()

sess = tf.InteractiveSession(config=config) #启动创建的模型
sess.run(tf.global_variables_initializer()) #初始化变量

# 开启一个训练
step = 1
# Keep training until reach max iterations
while step * batch_size < training_iters:
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    # 获取批数据
    sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys, keep_prob: dropout})
    if step % display_step == 0:
        # 计算精度
        acc = sess.run(accuracy, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
        # 计算损失值
        loss = sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
        print ("Iter " + str(step*batch_size) + ", Minibatch Loss= " + "{:.6f}".format(loss) + ", Training Accuracy = " + "{:.5f}".format(acc))
    step += 1
print ("Optimization Finished!")

# 计算测试精度
print ("Testing Accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images[:256], y: mnist.test.labels[:256], keep_prob: 1.}))

saver.save(sess, './model/mymodel', step*batch_size)

end = time.clock() #计算程序结束时间
print(("running time is %g s") % (end-start))

sess.close()

Extracting /MNIST_data\train-images-idx3-ubyte.gz
Extracting /MNIST_data\train-labels-idx1-ubyte.gz
Extracting /MNIST_data\t10k-images-idx3-ubyte.gz
Extracting /MNIST_data\t10k-labels-idx1-ubyte.gz


ValueError: At least two variables have the same name: Variable/Adam

In [1]:
# AlexNet 8 (Test from saved model)

#coding=utf-8
from __future__ import print_function
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

start = time.clock() #计算开始时间

mnist = input_data.read_data_sets("/MNIST_data", one_hot=True)

# 定义网络超参数
learning_rate = 0.001
training_iters = 2000
batch_size = 64
display_step = 20

# 定义网络参数
n_input = 784 # 输入的维度
n_classes = 10 # 标签的维度
dropout = 0.8 # Dropout 的概率

# 占位符输入
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# 卷积操作
def conv2d(name, l_input, w, b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(l_input, w, strides=[1, 1, 1, 1], padding='SAME'),b), name=name)

# 最大下采样操作
def max_pool(name, l_input, k):
    return tf.nn.max_pool(l_input, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME', name=name)

# 归一化操作
def norm(name, l_input, lsize=4):
    return tf.nn.lrn(l_input, lsize, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name=name)

# 定义整个网络 
def alex_net(_X, _weights, _biases, _dropout):
    # 向量转为矩阵
    _X = tf.reshape(_X, shape=[-1, 28, 28, 1])

    # 卷积层
    conv1 = conv2d('conv1', _X, _weights['wc1'], _biases['bc1'])
    # 下采样层
    pool1 = max_pool('pool1', conv1, k=2)
    # 归一化层
    norm1 = norm('norm1', pool1, lsize=4)
    # Dropout
    norm1 = tf.nn.dropout(norm1, _dropout)

    # 卷积
    conv2 = conv2d('conv2', norm1, _weights['wc2'], _biases['bc2'])
    # 下采样
    pool2 = max_pool('pool2', conv2, k=2)
    # 归一化
    norm2 = norm('norm2', pool2, lsize=4)
    # Dropout
    norm2 = tf.nn.dropout(norm2, _dropout)

    # 卷积
    conv3 = conv2d('conv3', norm2, _weights['wc3'], _biases['bc3'])
    # 下采样
    pool3 = max_pool('pool3', conv3, k=2)
    # 归一化
    norm3 = norm('norm3', pool3, lsize=4)
    # Dropout
    norm3 = tf.nn.dropout(norm3, _dropout)

    # 全连接层，先把特征图转为向量
    dense1 = tf.reshape(norm3, [-1, _weights['wd1'].get_shape().as_list()[0]]) 
    dense1 = tf.nn.relu(tf.matmul(dense1, _weights['wd1']) + _biases['bd1'], name='fc1') 
    # 全连接层
    dense2 = tf.nn.relu(tf.matmul(dense1, _weights['wd2']) + _biases['bd2'], name='fc2') # Relu activation

    # 网络输出层
    out = tf.matmul(dense2, _weights['out']) + _biases['out']
    return out

# 存储所有的网络参数
weights = {
    'wc1': tf.Variable(tf.random_normal([3, 3, 1, 64])),
    'wc2': tf.Variable(tf.random_normal([3, 3, 64, 128])),
    'wc3': tf.Variable(tf.random_normal([3, 3, 128, 256])),
    'wd1': tf.Variable(tf.random_normal([4*4*256, 1024])),
    'wd2': tf.Variable(tf.random_normal([1024, 1024])),
    'out': tf.Variable(tf.random_normal([1024, 10]))
}
biases = {
    'bc1': tf.Variable(tf.random_normal([64])),
    'bc2': tf.Variable(tf.random_normal([128])),
    'bc3': tf.Variable(tf.random_normal([256])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'bd2': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# 构建模型
pred = alex_net(x, weights, biases, keep_prob)

# 定义损失函数和学习步骤
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred, labels = y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# 测试网络
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

config = tf.ConfigProto(
        device_count = {'GPU': 0}
    )

sess = tf.InteractiveSession(config=config) #启动创建的模型
sess.run(tf.global_variables_initializer()) #初始化变量

saver = tf.train.Saver()
#saver = tf.train.import_meta_graph('./model/mymodel.ckpt-200000.meta')
#读取上面训练好的模型参数
saver.restore(sess, './model/mymodel.ckpt-200000')
print ("Testing Accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images[:256], y: mnist.test.labels[:256], keep_prob: 1.}))
sess.close()

end = time.clock() #计算程序结束时间
print(("running time is %g s") % (end-start))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /MNIST_data\t10k-images-idx3-ubyte.gz
Extracting /MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

INFO:tensorflow:Restoring parameters from ./model/mymodel.ckpt-200000
Testing Accuracy: 0.957031
running 

In [27]:
#coding:utf-8
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data
import time

image_size = 784 #28*28
class_num = 10# 0~10

#共训练100000次
total_step = 100000
#每隔10×batch_size步显示一次结果
display_step = 10
#学习率
learning_rate = 0.01
#每次找出50张图片进行训练
batch_size = 50

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True) #MNIST数据输入

#image的placehol der
x = tf.placeholder(tf.float32, [None,image_size])
#label的placeholder
y = tf.placeholder(tf.float32, [None,class_num])
#dropout的placeholder
dropoutP = tf.placeholder(tf.float32)

#卷积层定义
def conv2d(image_input,w,b,name):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(image_input, w, strides=[1, 1, 1, 1], padding='SAME'),b), name=name)

#下采样层定义
def pooling(featuremaps,kernel_size,name):
    return tf.nn.max_pool(featuremaps, [1,kernel_size,kernel_size,1], [1,kernel_size,kernel_size,1], padding='SAME')

#归一化操作
def normlize(featuremaps,l_size,name):
    return tf.nn.lrn(featuremaps, 4, bias=1, alpha=0.0001, beta=0.75)

#初始化参数
weights = {
           #[3,3,1,64]分别代表3*3*1的kernel，输入层有1个feature maps，输出层共64个feature maps
           'wc1' : tf.Variable(tf.random_normal([3,3,1,64])),
           #[3,3,64,128]分别代表3*3*64的kernel，输入层有64个feature maps，输出层有128个feature maps
           'wc2' : tf.Variable(tf.random_normal([3,3,64,128])),
           'wc3' : tf.Variable(tf.random_normal([3,3,128,256])),
           'wc4' : tf.Variable(tf.random_normal([2,2,256,512])),
           #全连接层的参数个数设定的原则好像是经过卷积层运算以后feature map的大小并没有发生改变
           #发生改变的原因都是pooling层28/2/2/2/2 = 2(7/2可能是为4）
           'wd1' : tf.Variable(tf.random_normal([2*2*512,1024])),
           'wd2' : tf.Variable(tf.random_normal([1024,1024])),
           'out' : tf.Variable(tf.random_normal([1024,10]))
           }
#初始化偏置项
biases = {
          'bc1' : tf.Variable(tf.random_normal([64])),
          'bc2' : tf.Variable(tf.random_normal([128])),
          'bc3' : tf.Variable(tf.random_normal([256])),
          'bc4' : tf.Variable(tf.random_normal([512])),
          'bd1' : tf.Variable(tf.random_normal([1024])),
          'bd2' : tf.Variable(tf.random_normal([1024])),
          'out' : tf.Variable(tf.random_normal([10]))
          }
#构建网络
def constructNet(images,weights,biases,_dropout):
    #首先把图片转为28*28*1的tensor
    images = tf.reshape(images,[-1,28,28,1])

    #第一个卷积层conv1
    conv1 = conv2d(images, weights['wc1'], biases['bc1'], 'conv1')
#    print 'conv1: ',conv1.get_shape()
    #卷积层conv1对应下采样层
    pool1 = pooling(conv1, 2, 'pool1')
#    print 'pool1: ',pool1.get_shape()
    #归一化
    norm1 = normlize(pool1, l_size=4, name='norm1')
    dropout1 = tf.nn.dropout(norm1, _dropout)

    #第二个卷积层
    conv2 = conv2d(dropout1,weights['wc2'],biases['bc2'],'conv2')
#    print 'conv2: ',conv2.get_shape()
    pool2 = pooling(conv2, 2, 'pool2')
#    print 'pool2: ',pool2.get_shape()
    norm2 = normlize(pool2, 4, 'norm2')
    dropout2 = tf.nn.dropout(norm2,_dropout)

    #第三个卷积层
    conv3 = conv2d(dropout2, weights['wc3'], biases['bc3'], 'conv3')
#    print 'conv3: ',conv3.get_shape()
    pool3 = pooling(conv3, 2, 'pool3')
#    print 'pool3: ',pool3.get_shape()
    norm3 = normlize(pool3, 4, 'norm3')
    dropout3 = tf.nn.dropout(norm3,_dropout)

    #第四个卷积层
    conv4 = conv2d(dropout3,weights['wc4'],biases['bc4'],'conv4')
#    print 'conv4: ',conv4.get_shape()
    pool4 = pooling(conv4, 2, 'pool4')
#    print 'pool4: ',pool4.get_shape()
    norm4 = normlize(pool4, 4, 'norm4')
#    print 'norm4: ',norm4.get_shape()
    #全链接层1
    dense1 = tf.reshape(norm4, [-1,weights['wd1'].get_shape().as_list()[0]])
    dense1 = tf.nn.relu(tf.matmul(dense1,weights['wd1']) + biases['bd1'],'fc1')
    #全链接层2
    dense2 = tf.nn.relu(tf.matmul(dense1,weights['wd2']) + biases['bd2'],'fc2')
    #输出层，最后输出层不需要激活函数relu操作
    out = tf.matmul(dense2,weights['out']) + biases['out']
    return out

pred = constructNet(x, weights, biases, dropoutP)
#计算loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred, labels = y))
#定义操作，用以最小化loss（Adam是一种梯度下降算法）
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#tf.arg_max(pred,1)是按行取最大值的下标
#tf.arg_max(y,1)是按列取最大值的下标
correct_pred = tf.equal(tf.arg_max(pred,1), tf.arg_max(y,1))
#先将correct_pred中数据格式转换为float32类型
#求correct_pred中的平均值，因为correct_pred中除了0就是1，因此求平均值即为1的所占比例，即正确率
correct_rate = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

config = tf.ConfigProto(
        device_count = {'GPU': 0}
    )

sess = tf.InteractiveSession(config=config) #启动创建的模型
sess.run(tf.global_variables_initializer()) #初始化变量

init = tf.initialize_all_variables()
saver = tf.train.Saver()

step = 1
while step*batch_size < total_step:
    batchx,batchy = mnist.train.next_batch(batch_size)
    sess.run(optimizer,feed_dict={x:batchx,y:batchy,dropoutP:0.75})
    if(step % display_step == 0):
        accracy = sess.run(correct_rate,feed_dict={x:batchx,y:batchy,dropoutP:1.})
        cost = sess.run(loss,feed_dict={x:batchx,y:batchy,dropoutP:1.})
        print("step: %d, cost: %g, train_accuracy: %g" %(step*batch_size, cost, accracy))
#            print 'Step: ' + str(step*batch_size) + ' cost: ' + str(cost) + ' accracy: ' + str(accracy)
        #保存当前网络的参数，以便测试时读取训练结果
        saver.save(sess, '/model/steven.netmodel',step)

    step += 1
#    print 'train Finished'

#读取上面训练好的模型参数
saver.restore(sess, '/model/steven.netmodel-1990')
#    print 'Testing accary: ',sess.run(correct_rate,feed_dict={x:mnist.test.images[:100],y:mnist.test.labels[:100],dropoutP:1.})

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `argmax` instead


C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


KeyboardInterrupt: 